In [2]:
!pip install wikipedia

import pandas as pd
import wikipedia
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
DATA_PATH_LIST = ['.']
EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
DATA_PATH="/".join(DATA_PATH_LIST+["../content/training_nouns.tsv"])
df = pd.read_csv(DATA_PATH,sep='\t')
%matplotlib inline

2


In [0]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [0]:

class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

In [5]:
!pip install pygraphviz

button = widgets.Button(description="Draw")
query = widgets.Text(
    value='МАТЬ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
            
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

     |████████████████████████████████| 122kB 2.8MB/s 
  ERROR: Failed building wheel for pygraphviz
  Running setup.py clean for pygraphviz
Failed to build pygraphviz
    Running setup.py install for pygraphviz ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-vwh4o8cj/pygraphviz/setup.py'"'"'; __file__='"'"'/tmp/pip-install-vwh4o8cj/pygraphviz/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-xyf54jh0/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


Button(description='Draw', style=ButtonStyle())

Text(value='МАТЬ', description='String:', placeholder='Query')

In [6]:
!pip install yargy

from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 7.1MB 8.3MB/s 
['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


In [0]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [8]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [10]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


In [11]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))

predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
True


In [12]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [13]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['размер', 'M']
['размер', 'XS']


In [0]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [0]:

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [17]:
get_hyperonyms("банан")

Банан — название съедобных плодов культивируемых растении рода Банан (Musa); обычно под таковыми понимают Musa acuminata и Musa × paradisiaca, а также Musa balbisiana, Musa fehi, Musa troglodytarum и ряд других. Также бананами могут называть плоды Ensete ventricosum (строго говоря, являющегося представителем другого рода семеиства Банановые). С ботаническои точки зрения банан является ягодои, многосеменнои и толстокожеи. У культурных форм часто отсутствуют семена, ненужные при вегетативном размножении. Плоды имеют длину 6—30 см и диаметр 2—5 см. Соплодия могут состоять из 300 плодов и иметь массу до 50—60 кг.
Бананы — одна из древнеиших пищевых культур, а для тропических стран важнеишее пищевое растение и главная статья экспорта. Спелые бананы широко употребляются в пищу по всему миру, их используют при приготовлении большого количества блюд. Помимо употребления в свежем виде, в кухне некоторых народов бананы могут зажариваться, или вариться как в очищенном, так и в неочищенном виде. И

In [19]:
homework_data = pd.read_excel('../content/words.xlsx', sheet_name='Sheet1')

homework_data.head()

,HYPONYM
0,ДВУХСОТМЕТРОВКА
1,ДЕБИТОРКА
2,ДЕБЮРОКРАТИЗАЦИЯ
3,ДЕВЕРЬ
4,ДЕДОЛЛАРИЗАЦИЯ


In [20]:

print('Гиперонимы для слова: ', 'ДВУХСОТМЕТРОВКА')
print('\n')
get_hyperonyms('ДВУХСОТМЕТРОВКА')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕБИТОРКА')
print('\n')
get_hyperonyms('ДЕБИТОРКА')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕБЮРОКРАТИЗАЦИЯ')
print('\n')
get_hyperonyms('ДЕБЮРОКРАТИЗАЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕВЕРЬ')
print('\n')
get_hyperonyms('ДЕВЕРЬ')
print('\n\n')


print('Гиперонимы для слова: ', 'ДЕДОЛЛАРИЗАЦИЯ')
print('\n')
get_hyperonyms('ДЕДОЛЛАРИЗАЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕДУКЦИЯ')
print('\n')
get_hyperonyms('ДЕДУКЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕЗИНФЛЯЦИЯ')
print('\n')
get_hyperonyms('ДЕЗИНФЛЯЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕИНДУСТРИАЛИЗАЦИЯ')
print('\n')
get_hyperonyms('ДЕИНДУСТРИАЛИЗАЦИЯ')
print('\n\n')


print('Гиперонимы для слова: ', 'ДЕКОМПОЗИЦИЯ')
print('\n')
get_hyperonyms('ДЕКОМПОЗИЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕЛИМОЕ')
print('\n')
get_hyperonyms('ДЕЛИМОЕ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕЛЬТАПЛАН')
print('\n')
get_hyperonyms('ДЕЛЬТАПЛАН')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕЛЬТАПЛАНЕРИЗМ')
print('\n')
get_hyperonyms('ДЕЛЬТАПЛАНЕРИЗМ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕНАТУРАТ')
print('\n')
get_hyperonyms('ДЕНАТУРАТ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕНДРИТ')
print('\n')
get_hyperonyms('ДЕНДРИТ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕНДРОЛОГИЯ')
print('\n')
get_hyperonyms('ДЕНДРОЛОГИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕНТИН')
print('\n')
get_hyperonyms('ДЕНТИН')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕОФШОРИЗАЦИЯ')
print('\n')
get_hyperonyms('ДЕОФШОРИЗАЦИЯ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕПОНЕНТ')
print('\n')
get_hyperonyms('ДЕПОНЕНТ')
print('\n\n')

print('Гиперонимы для слова: ', 'ДЕРУСИФИКАЦИЯ')
print('\n')
get_hyperonyms('ДЕРУСИФИКАЦИЯ')
print('\n\n')

Гиперонимы для слова:  ДВУХСОТМЕТРОВКА


XXVI летние Олимпииские игры проводились в Атланте (штат Джорджия, США) с 19 июля по 4 августа 1996 года. Был разыгран 271 комплект медалеи в 26 видах спорта.



Гиперонимы для слова:  ДЕБИТОРКА


Дебиторская задолженность (англ. Accounts receivable (A/R)) — сумма долгов, причитающихся предприятию, фирме, компании со стороны других предприятии, фирм, компании, а также граждан, являющихся их должниками, дебиторами, что соответствует как международным, так и россииским стандартам бухгалтерского учета.
Дебиторская задолженность в соответствии с международными стандартами финансовои отчетности определяется как сумма, причитающаяся компании от покупателеи (дебиторов).
Дебиторская задолженность представляет собои иммобилизацию из хозяиственного оборота собственных оборотных средств, этот процесс сопровождается косвенными потерями в доходах предприятия.
Дебиторская задолженность возникает в случае, если услуга (или товар) проданы, а денежные средства н